In [3]:
import pandas as pd
import pandas_datareader as pdr

intc_df = pdr.get_data_yahoo('INTC',start='2000-01-01')
sox_df = pdr.get_data_yahoo('^SOX',start='2000-01-01')
vix_df = pdr.get_data_yahoo('^VIX',start='2000-01-01')
snp500_df = pdr.get_data_yahoo('^GSPC',start='2000-01-01')

intc_df.to_csv('intc.csv')
sox_df.to_csv('sox.csv')
vix_df.to_csv('vix.csv')
snp500_df.to_csv('snp500.csv')

In [2]:
import talib
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import math
from sklearn.preprocessing import MinMaxScaler

In [5]:
df = pd.read_csv('intc.csv',index_col='Date',parse_dates=True)
sox_df = pd.read_csv('sox.csv',index_col='Date',parse_dates=True)
vix_df = pd.read_csv('vix.csv',index_col='Date',parse_dates=True)
snp500_df = pd.read_csv('snp500.csv',index_col='Date',parse_dates=True)

In [6]:
df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
1999-12-31,41.9375,40.96875,41.906250,41.15625,11572000.0,26.049402
2000-01-03,43.6875,41.62500,41.632812,43.50000,57710200.0,27.532846
2000-01-04,43.9375,41.12500,42.718750,41.46875,51019600.0,26.247194
2000-01-05,42.9375,40.25000,41.500000,41.81250,52389000.0,26.464764
2000-01-06,41.6875,39.09375,40.875000,39.37500,55171200.0,24.921970


In [8]:
df['next_rtn'] = df['Close']/df['Open'] - 1
df['log_return'] = np.log(1+df['Adj Close'].pct_change())

#이동 평균(Moving Average)
df['MA5'] = talib.SMA(df['Close'],timeperiod=5)
df['MA10'] = talib.SMA(df['Close'],timeperiod=10)
df['RASD5'] = talib.SMA(talib.STDDEV(df['Close'],timeperiod=5,nbdev=1),timeperiod=5)
df['RASD10'] = talib.SMA(talib.STDDEV(df['Close'],timeperiod=5,nbdev=1),timeperiod=10)

In [10]:
df.tail()

,High,Low,Open,Close,Volume,Adj Close,next_rtn,log_return,MA5,MA10,RASD5,RASD10
Date,,,,,,,,,,,,
2020-09-24,49.639999,48.419998,48.529999,49.160000,29343400.0,49.160000,0.012982,0.006940,49.508000,49.692,0.358343,0.380841
2020-09-25,50.279999,48.750000,48.959999,49.939999,26633400.0,49.939999,0.020016,0.015742,49.518000,49.758,0.379463,0.382337
2020-09-28,51.459999,50.160000,50.509998,51.430000,29652200.0,51.430000,0.018214,0.029399,49.860000,49.960,0.509541,0.445437
2020-09-29,51.740002,50.950001,51.310001,51.189999,19558200.0,51.189999,-0.002339,-0.004677,50.107999,50.079,0.668804,0.515662
2020-09-30,52.380001,51.150002,51.240002,51.779999,27698300.0,51.779999,0.010539,0.011460,50.699999,50.220,0.766571,0.563533


In [11]:
#MACD (Moving Average Convergence & Divergence)
macd, macdsignal, macdhist = talib.MACD(df['Close'],fastperiod=12,slowperiod=26,signalperiod=9)
df['MACD'] = macd

In [12]:
df.tail()

,High,Low,Open,Close,Volume,Adj Close,next_rtn,log_return,MA5,MA10,RASD5,RASD10,MACD
Date,,,,,,,,,,,,,
2020-09-24,49.639999,48.419998,48.529999,49.160000,29343400.0,49.160000,0.012982,0.006940,49.508000,49.692,0.358343,0.380841,-0.338432
2020-09-25,50.279999,48.750000,48.959999,49.939999,26633400.0,49.939999,0.020016,0.015742,49.518000,49.758,0.379463,0.382337,-0.288958
2020-09-28,51.459999,50.160000,50.509998,51.430000,29652200.0,51.430000,0.018214,0.029399,49.860000,49.960,0.509541,0.445437,-0.128043
2020-09-29,51.740002,50.950001,51.310001,51.189999,19558200.0,51.189999,-0.002339,-0.004677,50.107999,50.079,0.668804,0.515662,-0.019656
2020-09-30,52.380001,51.150002,51.240002,51.779999,27698300.0,51.779999,0.010539,0.011460,50.699999,50.220,0.766571,0.563533,0.112552


In [13]:
#모멘텀 지표
#CCI = Commodity Channel Index
df['CCI'] = talib.CCI(df['High'],df['Low'],df['Close'],timeperiod=14)

#변동성 지표
#ATR = Average True Range
df['ATR'] = talib.ATR(df['High'],df['Low'],df['Close'],timeperiod=14)

#볼린저 밴드
upper, middle, lower = talib.BBANDS(df['Close'],timeperiod=20,nbdevup=2,nbdevdn=2,matype=0)
df['ub'] = upper
df['middle'] = middle
df['lb'] = lower

#MTM1 MTM3
df['MTM1'] = talib.MOM(df['Close'],timeperiod=1)
df['MTM3'] = talib.MOM(df['Close'],timeperiod=3)

In [14]:
df.tail()

,High,Low,Open,Close,Volume,Adj Close,next_rtn,log_return,MA5,MA10,RASD5,RASD10,MACD,CCI,ATR,ub,middle,lb,MTM1,MTM3
Date,,,,,,,,,,,,,,,,,,,,
2020-09-24,49.639999,48.419998,48.529999,49.160000,29343400.0,49.160000,0.012982,0.006940,49.508000,49.692,0.358343,0.380841,-0.338432,-99.095298,1.216431,51.542414,49.9350,48.327586,0.340000,-0.560001
2020-09-25,50.279999,48.750000,48.959999,49.939999,26633400.0,49.939999,0.020016,0.015742,49.518000,49.758,0.379463,0.382337,-0.288958,9.749435,1.238828,51.550592,49.9620,48.373408,0.779999,-0.010002
2020-09-28,51.459999,50.160000,50.509998,51.430000,29652200.0,51.430000,0.018214,0.029399,49.860000,49.960,0.509541,0.445437,-0.128043,201.957021,1.258912,51.715181,50.0120,48.308819,1.490002,2.610001
2020-09-29,51.740002,50.950001,51.310001,51.189999,19558200.0,51.189999,-0.002339,-0.004677,50.107999,50.079,0.668804,0.515662,-0.019656,181.023671,1.225418,51.756575,50.0240,48.291425,-0.240002,2.029999
2020-09-30,52.380001,51.150002,51.240002,51.779999,27698300.0,51.779999,0.010539,0.011460,50.699999,50.220,0.766571,0.563533,0.112552,178.900064,1.225746,51.942021,50.0735,48.204979,0.590000,1.840000


In [15]:
snp500_df = snp500_df.loc[:,['Close']].copy()
snp500_df.rename(columns={'Close':'S&P500'},inplace=True)
vix_df = vix_df.loc[:,['Close']].copy()
vix_df.rename(columns={'Close':'VIX'},inplace=True)
sox_df = sox_df.loc[:,['Close']].copy()
sox_df.rename(columns={'Close':'SOX'},inplace=True)

snp500_df.head()

,S&P500
Date,
1999-12-31,1469.250000
2000-01-03,1455.219971
2000-01-04,1399.420044
2000-01-05,1402.109985
2000-01-06,1403.449951


In [17]:
df =df.join(snp500_df,how='left')
df =df.join(sox_df,how='left')
df =df.join(vix_df,how='left')

In [24]:
print(df.columns)
df.tail()

Index(['High', 'Low', 'Open', 'Close', 'Volume', 'Adj Close', 'next_rtn',
       'log_return', 'MA5', 'MA10', 'RASD5', 'RASD10', 'MACD', 'CCI', 'ATR',
       'ub', 'middle', 'lb', 'MTM1', 'MTM3', 'S&P500', 'SOX', 'VIX'],
      dtype='object')


,High,Low,Open,Close,Volume,Adj Close,next_rtn,log_return,MA5,MA10,...,CCI,ATR,ub,middle,lb,MTM1,MTM3,S&P500,SOX,VIX
Date,,,,,,,,,,,,,,,,,,,,,
2020-09-24,49.639999,48.419998,48.529999,49.160000,29343400.0,49.160000,0.012982,0.006940,49.508000,49.692,...,-99.095298,1.216431,51.542414,49.9350,48.327586,0.340000,-0.560001,3246.590088,2147.419922,28.510000
2020-09-25,50.279999,48.750000,48.959999,49.939999,26633400.0,49.939999,0.020016,0.015742,49.518000,49.758,...,9.749435,1.238828,51.550592,49.9620,48.373408,0.779999,-0.010002,3298.459961,2177.840088,26.379999
2020-09-28,51.459999,50.160000,50.509998,51.430000,29652200.0,51.430000,0.018214,0.029399,49.860000,49.960,...,201.957021,1.258912,51.715181,50.0120,48.308819,1.490002,2.610001,3351.600098,2237.639893,26.190001
2020-09-29,51.740002,50.950001,51.310001,51.189999,19558200.0,51.189999,-0.002339,-0.004677,50.107999,50.079,...,181.023671,1.225418,51.756575,50.0240,48.291425,-0.240002,2.029999,3335.469971,2242.520020,26.270000
2020-09-30,52.380001,51.150002,51.240002,51.779999,27698300.0,51.779999,0.010539,0.011460,50.699999,50.220,...,178.900064,1.225746,51.942021,50.0735,48.204979,0.590000,1.840000,3363.000000,2244.120117,26.370001


In [25]:
df.to_csv('df.csv')